# Tensorflow 运行机制
## 介绍

这里所用的环境如下：
1. python 2.7
2. tensorflow 1.6.0

3. GPU 1080TI x2


### 通用Tensorflow 算法框架
这里我们将介绍通用的tensorflow算法流程。大多数算法都采用这种框架。

1. ** 准备数据（导入或者产生）**

    所有的机器学习算法都依赖于数据。在这本书中，我们使用的数据将来自于生成的数据或者外部数据源。有时候使用生成的数据会更好，因为我们总是想知道正确的结果应该是怎样的。其他情况下，我们会使用外部数据，获取这些数据的具体细节请看这个章节的第8小节。

2. **数据转换和归一化**

    原始数据往往不是以Tensorflow算法所需要的纬度或格式存在，因此我们将在使用数据之前对他们进行转换。大多数算法需要归一化的输入数据，我们也会在这个步骤中进行处理，并且Tensorflow已经为你准备好了数据归一化的方法。

    > `data = tf.nn.batch_norm_with_global_normalization(...)`

3. **设置模型超参数**

    我们的机器学习算法往往会包含一系列的超参数。例如，算法迭代次数，学习率，或者其他的可选参数。将他们一起初始化是一种很好的选择，因为这样其他的读者可以很容易找到他们并进行修改。

    > `learning_rate = 0.01`
    > `iterations = 1000`

4. **初始化variables和placeholders**（强行翻译的话，就是变量和占位符）

    Tensorflow需要我们告诉它，什么量是常量，什么量是变量。Tensorflow将会在模型训练过程中修改variables的值，直到最小化loss或者迭代结束。在模型训练过程中，我们通过placeholder将数据“喂”给模型。我们需要同时初始化variables和placeholders并且指定他们的格式和类别，这样Tensorflow就直到该接受什么样的数据。

    ps：我对placeholder的理解就是，placeholder就是用来占坑的，在模型训练之前，占好对应计算位置的坑，然后在模型训练过程中，动态往坑里塞对应的内容，然后就可以正常训练了。

    > `a_var = tf.constant(42)`

    > `x_input = tf.placeholder(tf.float32, [None, input_size])`

    > `y_input = tf.placeholder(tf.float32, [None, num_classes])`

    > `weight = tf.get_variable('weight', shape=[3,3,3])`

5. **定义模型框架**

    在我们准备好数据，并初始化各种参数、variables以及placeholders之后，我们需要定义我们的模型。这个步骤通过构建一个`computational graph（计算图）`来完成。我们需要告诉tensorlfow应该对variables和placeholders进行什么样对操作来得到我们想要的预测结果。我们将在这本书对第2章第1小节节深入讨论。

6. **定义loss function**（损失函数）

    在定义好模型之后，我们需要对模型对输出进行评价。在这里我们需要定义损失函数。损失函数非常重要，它用来告诉我们，我们目前的模型输出的结果离真实结果有多“远”。各种损失函数的细节将在本书的第2章节，第5小节详细提及。

    > `loss = tf.reduce_mean(tf.square(y_actual - y_pred))`

7. **初始化并训练模型**

    经过以上5个步骤，所有的东西已经就位。接下来，我们创建一个实例或者说一个图并且通过placeholders将我们的数据“喂”给模型，然后让Tensorflow来改变vairables以获得模型在训练数据上更好的训练结果。下面给出一种初始化计算图的方式：

    > ```
    with tf.Session(graph=graph) as sess:
    ...
    sess.run(...)
    ...
    ```

    我们也可以通过下面的方式初始化计算图：

    > ```
    sess = tf.Session(graph=graph)
    sess.run(...)
    ```

8. **（可选）评价模型**

    一旦我们训练好了一个模型之后，我们可以通过一些特定的评价指标，比如precision、recall等等来评价模型在新的数据上的表现。

9. **（可选）预测方法**

    构建预测方法也是非常重要的，我们可以通过预测方法来使用已经训练好的模型对新的、没有见过的数据进行预测。这也是模型的价值所在。


### 总结

在Tensorflow中，我们先设置好数据、variables、placeholders，并定义好模型，然后告诉程序进行训练并让tensorflow改变variables来提高预测结果。Tensoflow通过计算图来完成这个过程。我们告诉它最小化loss function，Tensorflow通过改变variables的值来实现。Tensorflow知道如何更新variables是因为它在训练过程中对模型中计算过程保持监控，并自动计算每个variable的梯度。正因为如此，我们可以很简单对模型进行改变或者尝试新的数据。

总的来说，Tensorflow中算法被设计为可循环的。我们构建这样的循环作为一个计算图，然后（1）将数据通过placeholders“喂”给模型；（2）计算计算图的输出；（3）通过loss function来比较数据和真实结果的差距；（4）通过返向传播算法改变模型的variables；（5）重复整个过程直到达到停止标准（如：达到迭代次数）。



![tensorflow_process_diagram](./images/01_outline.png)